# Using Redis for Embeddings Search

This notebook takes you through a simple flow to download some data, embed it, and then index and search it using a selection of vector databases. This is a common requirement for customers who want to store and search our embeddings with their own data in a secure environment to support production use cases such as chatbots, topic modelling and more.

### What is a Vector Database

A vector database is a database made to store, manage and search embedding vectors. The use of embeddings to encode unstructured data (text, audio, video and more) as vectors for consumption by machine-learning models has exploded in recent years, due to the increasing effectiveness of AI in solving use cases involving natural language, image recognition and other unstructured forms of data. Vector databases have emerged as an effective solution for enterprises to deliver and scale these use cases.

### Why use a Vector Database

Vector databases enable enterprises to take many of the embeddings use cases we've shared in this repo (question and answering, chatbot and recommendation services, for example), and make use of them in a secure, scalable environment. Many of our customers make embeddings solve their problems at small scale but performance and security hold them back from going into production - we see vector databases as a key component in solving that, and in this guide we'll walk through the basics of embedding text data, storing it in a vector database and using it for semantic search.


### Demo Flow
The demo flow is:
- **Setup**: Import packages and set any required variables
- **Load data**: Load a dataset and embed it using OpenAI embeddings
- **Redis**
    - *Setup*: Set up the Redis-Py client. For more details go [here](https://github.com/redis/redis-py)
    - *Index Data*: Create the search index for vector search and hybrid search (vector + full-text search) on all available fields.
    - *Search Data*: Run a few example queries with various goals in mind.

Once you've run through this notebook you should have a basic understanding of how to setup and use vector databases, and can move on to more complex use cases making use of our embeddings.

## Setup

Import required libraries and load the OpenAI API key from `redis_config.json`.

<details>
<summary>💡 <b>What's happening here?</b> (click to expand)</summary>

<br>

**Libraries are like tools:**
- `pandas` = data tables
- `numpy` = math operations  
- `redis` = talk to our database
- `openai` = create embeddings (convert text to vectors)

**API Key** = Your ID card to use OpenAI's service

**EMBEDDING_MODEL** = `text-embedding-3-small` converts text into 1536 numbers

**Why redis_config.json?** Keeps your secret API key out of the code (secure!)

</details>

In [1]:
import openai

from typing import List, Iterator
import pandas as pd
import numpy as np
import os
import wget
from ast import literal_eval

# Redis client library for Python
import redis

# Load configuration (including OpenAI API key)
import json

config_file = 'redis_config.json'
if os.path.exists(config_file):
    with open(config_file, 'r') as f:
        config = json.load(f)
    # Set OpenAI API key from config file
    if 'openai_api_key' in config:
        openai.api_key = config['openai_api_key']
        print("✅ OpenAI API key loaded from config file")
    else:
        openai.api_key = os.getenv("OPENAI_API_KEY", "")
        print("⚠️  OpenAI API key not in config, using environment variable")
else:
    openai.api_key = os.getenv("OPENAI_API_KEY", "")
    print("⚠️  Config file not found, using environment variable for OpenAI key")

# I've set this to our new embeddings model, this can be changed to the embedding model of your choice
EMBEDDING_MODEL = "text-embedding-3-small"

# Ignore unclosed SSL socket warnings - optional in case you get these errors
import warnings

warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

/Users/banbalagan/Projects/redis-vector-db-assignment/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


✅ OpenAI API key loaded from config file


## Load Data

Download the pre-embedded Wikipedia dataset (25K articles with vectors already computed). We'll use 100 articles to fit within the Redis Cloud free tier constraints (30MB memory + 510 vectors).

<details>
<summary>💡 <b>What's happening here?</b> (click to expand)</summary>

<br>

**The Dataset:**
- 25,000 Wikipedia articles
- Each article already has vectors (1536 numbers representing meaning)
- Pre-embedded = saves time and money!

**Why only 100 articles?**
- Redis Cloud free tier has 2 limits:
  - 30MB memory limit
  - 510 vectors per index limit
- We use 100 to comfortably fit both constraints
- Still enough to learn all vector database concepts!

**Vectors = Meaning:**
- Articles are represented as coordinates in 1536-dimensional space
- Similar articles = close together in this space
- Like comparing "summary cards" instead of reading entire books

</details>

In [2]:
# Step 1: Download the pre-embedded Wikipedia dataset (~700MB) - only if not already downloaded
embeddings_url = 'https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip'
zip_file = "vector_database_wikipedia_articles_embedded.zip"
csv_file = "../data/vector_database_wikipedia_articles_embedded.csv"

if os.path.exists(csv_file):
    print(f"✅ Data already exists at {csv_file} - skipping download")
elif os.path.exists(zip_file):
    print(f"✅ Zip file already exists - skipping download")
else:
    print("📥 Downloading dataset (this may take a few minutes)...")
    wget.download(embeddings_url)
    print("\n✅ Download complete!")

# Step 2: Extract the zip file (only if CSV doesn't exist)
if not os.path.exists(csv_file):
    if os.path.exists(zip_file):
        print("\n📦 Extracting files...")
        import zipfile
        with zipfile.ZipFile(zip_file, "r") as zip_ref:
            zip_ref.extractall("../data")
        print("✅ Files extracted to ../data/")
    else:
        print("❌ Zip file not found - please check download")
else:
    print("✅ CSV already extracted - skipping extraction")

# Step 3: Load the CSV into a pandas DataFrame
print("\n📊 Loading data into DataFrame...")
article_df = pd.read_csv(csv_file)
print(f"✅ Loaded {len(article_df)} articles")

# Step 4: Preview the data
print("\n👀 First 5 articles:")
display(article_df.head())

✅ Data already exists at ../data/vector_database_wikipedia_articles_embedded.csv - skipping download
✅ CSV already extracted - skipping extraction

📊 Loading data into DataFrame...


✅ Loaded 25000 articles

👀 First 5 articles:


,id,url,title,text,title_vector,content_vector,vector_id
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...,"[0.001009464613161981, -0.020700545981526375, ...","[-0.011253940872848034, -0.013491976074874401,...",0
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...,"[0.0009286514250561595, 0.000820168002974242, ...","[0.0003609954728744924, 0.007262262050062418, ...",1
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imag...,"[0.003393713850528002, 0.0061537534929811954, ...","[-0.004959689453244209, 0.015772193670272827, ...",2
3,8,https://simple.wikipedia.org/wiki/A,A,A or a is the first letter of the English alph...,"[0.0153952119871974, -0.013759135268628597, 0....","[0.024894846603274345, -0.022186409682035446, ...",3
4,9,https://simple.wikipedia.org/wiki/Air,Air,Air refers to the Earth's atmosphere. Air is a...,"[0.02224554680287838, -0.02044147066771984, -0...","[0.021524671465158463, 0.018522677943110466, -...",4


In [3]:
# Step 5: Prepare the vectors for Redis
print("\n🔧 Preparing vectors...")

# Convert vector strings to actual Python lists
# They're currently stored as text like "[0.1, 0.2, ...]" and need to be real lists
article_df['title_vector'] = article_df.title_vector.apply(literal_eval)
article_df['content_vector'] = article_df.content_vector.apply(literal_eval)

# Convert vector_id to string (Redis expects string keys)
article_df['vector_id'] = article_df['vector_id'].apply(str)

# Step 6: Reduce dataset for Redis Cloud Free Tier
# Free tier has both memory (30MB) and vector (510) limits
# We'll use 100 articles to comfortably fit within both constraints
print(f"📚 Full dataset size: {len(article_df)} articles")
article_df = article_df.head(100)  # Use 100 articles
print(f"✅ Using {len(article_df)} articles (fits in Redis Cloud free tier)")
print(f"💡 Free tier: 30MB memory + 510 vector limit - using 100 articles to stay safe")

# Step 7: Show dataset info
print("\n📋 Dataset Information:")
article_df.info(show_counts=True)


🔧 Preparing vectors...


📚 Full dataset size: 25000 articles
✅ Using 100 articles (fits in Redis Cloud free tier)
💡 Free tier: 30MB memory + 510 vector limit - using 100 articles to stay safe

📋 Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              100 non-null    int64 
 1   url             100 non-null    object
 2   title           100 non-null    object
 3   text            100 non-null    object
 4   title_vector    100 non-null    object
 5   content_vector  100 non-null    object
 6   vector_id       100 non-null    object
dtypes: int64(1), object(6)
memory usage: 5.6+ KB


# Redis

The next vector database covered in this tutorial is **[Redis](https://redis.io)**. You most likely already know Redis. What you might not be aware of is the [RediSearch module](https://github.com/RediSearch/RediSearch). Enterprises have been using Redis with the RediSearch module for years now across all major cloud providers, Redis Cloud, and on premise. Recently, the Redis team added vector storage and search capability to this module in addition to the features RediSearch already had.

Given the large ecosystem around Redis, there are most likely client libraries in the language you need. You can use any standard Redis client library to run RediSearch commands, but it's easiest to use a library that wraps the RediSearch API. Below are a few examples, but you can find more client libraries [here](https://redis.io/resources/clients/).

| Project | Language | License | Author | Stars |
|----------|---------|--------|---------|-------|
| [jedis][jedis-url] | Java | MIT | [Redis][redis-url] | ![Stars][jedis-stars] |
| [redis-py][redis-py-url] | Python | MIT | [Redis][redis-url] | ![Stars][redis-py-stars] |
| [node-redis][node-redis-url] | Node.js | MIT | [Redis][redis-url] | ![Stars][node-redis-stars] |
| [nredisstack][nredisstack-url] | .NET | MIT | [Redis][redis-url] | ![Stars][nredisstack-stars] |
| [redisearch-go][redisearch-go-url] | Go | BSD | [Redis][redisearch-go-author] | [![redisearch-go-stars]][redisearch-go-url] |
| [redisearch-api-rs][redisearch-api-rs-url] | Rust | BSD | [Redis][redisearch-api-rs-author] | [![redisearch-api-rs-stars]][redisearch-api-rs-url] |

[redis-url]: https://redis.com

[redis-py-url]: https://github.com/redis/redis-py
[redis-py-stars]: https://img.shields.io/github/stars/redis/redis-py.svg?style=social&amp;label=Star&amp;maxAge=2592000
[redis-py-package]: https://pypi.python.org/pypi/redis

[jedis-url]: https://github.com/redis/jedis
[jedis-stars]: https://img.shields.io/github/stars/redis/jedis.svg?style=social&amp;label=Star&amp;maxAge=2592000
[Jedis-package]: https://search.maven.org/artifact/redis.clients/jedis

[nredisstack-url]: https://github.com/redis/nredisstack
[nredisstack-stars]: https://img.shields.io/github/stars/redis/nredisstack.svg?style=social&amp;label=Star&amp;maxAge=2592000
[nredisstack-package]: https://www.nuget.org/packages/nredisstack/

[node-redis-url]: https://github.com/redis/node-redis
[node-redis-stars]: https://img.shields.io/github/stars/redis/node-redis.svg?style=social&amp;label=Star&amp;maxAge=2592000
[node-redis-package]: https://www.npmjs.com/package/redis

[redis-om-python-url]: https://github.com/redis/redis-om-python
[redis-om-python-author]: https://redis.com
[redis-om-python-stars]: https://img.shields.io/github/stars/redis/redis-om-python.svg?style=social&amp;label=Star&amp;maxAge=2592000

[redisearch-go-url]: https://github.com/RediSearch/redisearch-go
[redisearch-go-author]: https://redis.com
[redisearch-go-stars]: https://img.shields.io/github/stars/RediSearch/redisearch-go.svg?style=social&amp;label=Star&amp;maxAge=2592000

[redisearch-api-rs-url]: https://github.com/RediSearch/redisearch-api-rs
[redisearch-api-rs-author]: https://redis.com
[redisearch-api-rs-stars]: https://img.shields.io/github/stars/RediSearch/redisearch-api-rs.svg?style=social&amp;label=Star&amp;maxAge=2592000


In the below cells, we will walk you through using Redis as a vector database. Since many of you are likely already used to the Redis API, this should be familiar to most.

## Connect to Redis

Load credentials from `redis_config.json` and establish connection to Redis Cloud.

<details>
<summary>💡 <b>What's happening here?</b> (click to expand)</summary>

<br>

**Redis + RediSearch = Vector Database**

Regular Redis is just a fast key-value store, but with the RediSearch module it can:
- Store vectors efficiently
- Search by similarity (not just exact matches)
- Handle hybrid queries (vector + text filters)

**What we're doing:**
1. Load credentials (host, port, password) from config file
2. Connect to Redis Cloud 
3. Test connection with `ping()`

**Redis Cloud** = Managed service (like Gmail vs. running your own email server)

</details>

In [4]:
import redis
from redis.commands.search.indexDefinition import (
    IndexDefinition,
    IndexType
)
from redis.commands.search.query import Query
from redis.commands.search.field import (
    TextField,
    VectorField
)

# Load Redis connection details from config file
import json
import os

config_file = 'redis_config.json'
if os.path.exists(config_file):
    with open(config_file, 'r') as f:
        config = json.load(f)
    REDIS_HOST = config['redis_host']
    REDIS_PORT = config['redis_port']
    REDIS_PASSWORD = config['redis_password']
    print(f"✅ Loaded config from {config_file}")
else:
    # Fallback to environment variables or localhost
    REDIS_HOST = os.getenv('REDIS_HOST', 'localhost')
    REDIS_PORT = int(os.getenv('REDIS_PORT', 6379))
    REDIS_PASSWORD = os.getenv('REDIS_PASSWORD', '')
    print(f"⚠️  Config file not found. Using environment variables or defaults.")
    print(f"   Create {config_file} with your Redis Cloud credentials.")

# Connect to Redis
redis_client = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD
)
print("Testing Redis connection...")
response = redis_client.ping()
print(f"✅ Connected to Redis! Response: {response}")

✅ Loaded config from redis_config.json
Testing Redis connection...
✅ Connected to Redis! Response: True


## Health Check & Cleanup (Optional)

Check database status and optionally cleanup before proceeding. Useful when re-running the notebook.

Run `check_redis_health(redis_client)` to see current state, or `cleanup_redis(redis_client, 'index_and_docs')` to start fresh.

In [5]:
from IPython.display import display, HTML, Markdown

def check_redis_health(client: redis.Redis) -> dict:
    """
    Comprehensive Redis database health check with rich visual output.
    Returns a dictionary with health metrics and status.
    """
    health_status = {
        "connection": False,
        "memory_usage_mb": 0,
        "total_keys": 0,
        "indices": [],
        "doc_keys": 0,
        "memory_percent": 0,
        "warnings": []
    }
    
    # Header
    display(HTML("""
    <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                padding: 20px; border-radius: 10px; margin: 10px 0;'>
        <h2 style='color: white; margin: 0; text-align: center;'>
            🏥 REDIS DATABASE HEALTH CHECK
        </h2>
    </div>
    """))
    
    try:
        # Test connection
        client.ping()
        health_status["connection"] = True
        display(HTML("<div style='color: #10b981; font-weight: bold; font-size: 16px;'>✅ Redis connection is healthy</div>"))
        
        # Get server info
        info = client.info()
        memory_used = info.get('used_memory', 0) / (1024 * 1024)  # Convert to MB
        health_status["memory_usage_mb"] = round(memory_used, 2)
        
        # Get total keys
        for db_key in info.keys():
            if db_key.startswith('db'):
                health_status["total_keys"] += info[db_key].get('keys', 0)
        
        # Display metrics in styled boxes
        display(HTML(f"""
        <div style='display: flex; gap: 15px; margin: 20px 0; flex-wrap: wrap;'>
            <div style='background: #f0f9ff; border-left: 4px solid #3b82f6; 
                        padding: 15px; border-radius: 5px; flex: 1; min-width: 200px;'>
                <div style='color: #3b82f6; font-size: 24px;'>📊</div>
                <div style='font-size: 14px; color: #64748b; margin-top: 5px;'>Memory Usage</div>
                <div style='font-size: 24px; font-weight: bold; color: #1e293b;'>{health_status['memory_usage_mb']} MB</div>
            </div>
            <div style='background: #fef3c7; border-left: 4px solid #f59e0b; 
                        padding: 15px; border-radius: 5px; flex: 1; min-width: 200px;'>
                <div style='color: #f59e0b; font-size: 24px;'>🔑</div>
                <div style='font-size: 14px; color: #64748b; margin-top: 5px;'>Total Keys</div>
                <div style='font-size: 24px; font-weight: bold; color: #1e293b;'>{health_status['total_keys']}</div>
            </div>
        </div>
        """))
        
        # Check for existing indices
        try:
            indices = []
            # Try to get info on our specific index
            try:
                index_info = client.ft("embeddings-index").info()
                indices.append("embeddings-index")
                index_docs = index_info.get('num_docs', 0)
                display(HTML(f"""
                <div style='background: #dcfce7; border-left: 4px solid #10b981; 
                            padding: 15px; border-radius: 5px; margin: 10px 0;'>
                    <div style='color: #10b981; font-weight: bold;'>
                        📇 Index 'embeddings-index' exists with {index_docs} documents
                    </div>
                </div>
                """))
            except:
                display(HTML("""
                <div style='background: #f3f4f6; border-left: 4px solid #6b7280; 
                            padding: 15px; border-radius: 5px; margin: 10px 0;'>
                    <div style='color: #6b7280;'>
                        📇 Index 'embeddings-index' does not exist (will be created)
                    </div>
                </div>
                """))
            
            health_status["indices"] = indices
        except Exception as e:
            print(f"⚠️  Could not check indices: {e}")
        
        # Count document keys with our prefix
        doc_count = 0
        for key in client.scan_iter("doc:*"):
            doc_count += 1
            if doc_count >= 10:  # Sample only first 10 for performance
                # Get total count properly
                doc_count = len(list(client.scan_iter("doc:*")))
                break
        
        health_status["doc_keys"] = doc_count
        if doc_count > 0:
            display(HTML(f"""
            <div style='background: #e0e7ff; border-left: 4px solid #6366f1; 
                        padding: 15px; border-radius: 5px; margin: 10px 0;'>
                <div style='color: #6366f1; font-weight: bold;'>
                    📄 Found {doc_count} document keys with prefix 'doc:'
                </div>
            </div>
            """))
        
        # Check memory usage percentage (assuming 30MB free tier limit)
        FREE_TIER_LIMIT_MB = 30
        memory_percent = (memory_used / FREE_TIER_LIMIT_MB) * 100
        health_status["memory_percent"] = round(memory_percent, 1)
        
        # Memory usage gauge
        if memory_percent > 90:
            color = "#ef4444"
            bg_color = "#fee2e2"
            icon = "⚠️"
            message = f"WARNING: Using {memory_percent}% of free tier limit (30MB)"
            health_status["warnings"].append(message)
        elif memory_percent > 70:
            color = "#f59e0b"
            bg_color = "#fef3c7"
            icon = "⚠️"
            message = f"NOTICE: Using {memory_percent}% of free tier limit"
            health_status["warnings"].append(message)
        else:
            color = "#10b981"
            bg_color = "#dcfce7"
            icon = "✅"
            message = f"Memory usage is healthy ({memory_percent}% of 30MB free tier)"
        
        display(HTML(f"""
        <div style='background: {bg_color}; border-left: 4px solid {color}; 
                    padding: 15px; border-radius: 5px; margin: 15px 0;'>
            <div style='color: {color}; font-weight: bold; font-size: 16px;'>
                {icon} {message}
            </div>
            <div style='background: #e5e7eb; border-radius: 10px; height: 20px; 
                        margin-top: 10px; overflow: hidden;'>
                <div style='background: {color}; height: 100%; width: {memory_percent}%; 
                            transition: width 0.3s ease;'></div>
            </div>
        </div>
        """))
        
    except redis.exceptions.ConnectionError as e:
        health_status["connection"] = False
        health_status["warnings"].append(f"Connection Error: {e}")
        display(HTML(f"""
        <div style='background: #fee2e2; border-left: 4px solid #ef4444; 
                    padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <div style='color: #ef4444; font-weight: bold;'>
                ❌ Redis connection failed: {e}
            </div>
        </div>
        """))
    except Exception as e:
        health_status["warnings"].append(f"Health check error: {e}")
        display(HTML(f"""
        <div style='background: #fef3c7; border-left: 4px solid #f59e0b; 
                    padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <div style='color: #f59e0b; font-weight: bold;'>
                ⚠️  Error during health check: {e}
            </div>
        </div>
        """))
    
    return health_status


def cleanup_redis(client: redis.Redis, cleanup_type: str = "all"):
    """
    Clean up Redis database with rich visual feedback.
    
    Args:
        client: Redis client instance
        cleanup_type: Type of cleanup to perform
            - "all": Delete all keys (FLUSHDB)
            - "index": Delete only the search index
            - "docs": Delete only document keys (doc:*)
            - "index_and_docs": Delete index and document keys
    """
    display(HTML(f"""
    <div style='background: linear-gradient(135deg, #f59e0b 0%, #dc2626 100%); 
                padding: 15px; border-radius: 10px; margin: 10px 0;'>
        <h3 style='color: white; margin: 0;'>🧹 Starting Cleanup: {cleanup_type}</h3>
    </div>
    """))
    
    try:
        if cleanup_type == "all":
            # Nuclear option - delete everything
            confirm = input("⚠️  This will delete ALL data in the database. Type 'YES' to confirm: ")
            if confirm == "YES":
                client.flushdb()
                display(HTML("""
                <div style='background: #dcfce7; border-left: 4px solid #10b981; 
                            padding: 15px; border-radius: 5px; margin: 10px 0;'>
                    <div style='color: #10b981; font-weight: bold;'>
                        ✅ Database flushed completely
                    </div>
                </div>
                """))
            else:
                display(HTML("""
                <div style='background: #f3f4f6; border-left: 4px solid #6b7280; 
                            padding: 15px; border-radius: 5px; margin: 10px 0;'>
                    <div style='color: #6b7280;'>❌ Cleanup cancelled</div>
                </div>
                """))
                return
        
        elif cleanup_type == "index":
            # Delete only the search index
            try:
                client.ft("embeddings-index").dropindex(delete_documents=False)
                display(HTML("""
                <div style='background: #dcfce7; border-left: 4px solid #10b981; 
                            padding: 15px; border-radius: 5px; margin: 10px 0;'>
                    <div style='color: #10b981; font-weight: bold;'>
                        ✅ Search index 'embeddings-index' deleted (documents preserved)
                    </div>
                </div>
                """))
            except Exception as e:
                display(HTML(f"""
                <div style='background: #fef3c7; border-left: 4px solid #f59e0b; 
                            padding: 15px; border-radius: 5px; margin: 10px 0;'>
                    <div style='color: #f59e0b;'>
                        ⚠️  Could not delete index (may not exist): {e}
                    </div>
                </div>
                """))
        
        elif cleanup_type == "docs":
            # Delete only document keys
            doc_keys = list(client.scan_iter("doc:*"))
            if doc_keys:
                count = len(doc_keys)
                confirm = input(f"⚠️  This will delete {count} document keys. Type 'YES' to confirm: ")
                if confirm == "YES":
                    for key in doc_keys:
                        client.delete(key)
                    display(HTML(f"""
                    <div style='background: #dcfce7; border-left: 4px solid #10b981; 
                                padding: 15px; border-radius: 5px; margin: 10px 0;'>
                        <div style='color: #10b981; font-weight: bold;'>
                            ✅ Deleted {count} document keys
                        </div>
                    </div>
                    """))
                else:
                    display(HTML("""
                    <div style='background: #f3f4f6; border-left: 4px solid #6b7280; 
                                padding: 15px; border-radius: 5px; margin: 10px 0;'>
                        <div style='color: #6b7280;'>❌ Cleanup cancelled</div>
                    </div>
                    """))
            else:
                display(HTML("""
                <div style='background: #e0e7ff; border-left: 4px solid #6366f1; 
                            padding: 15px; border-radius: 5px; margin: 10px 0;'>
                    <div style='color: #6366f1;'>ℹ️  No document keys found</div>
                </div>
                """))
        
        elif cleanup_type == "index_and_docs":
            # Delete index AND documents
            try:
                client.ft("embeddings-index").dropindex(delete_documents=True)
                display(HTML("""
                <div style='background: #dcfce7; border-left: 4px solid #10b981; 
                            padding: 15px; border-radius: 5px; margin: 10px 0;'>
                    <div style='color: #10b981; font-weight: bold;'>
                        ✅ Search index 'embeddings-index' and all documents deleted
                    </div>
                </div>
                """))
            except Exception as e:
                display(HTML(f"""
                <div style='background: #fef3c7; border-left: 4px solid #f59e0b; 
                            padding: 15px; border-radius: 5px; margin: 10px 0;'>
                    <div style='color: #f59e0b;'>
                        ⚠️  Could not delete index: {e}
                    </div>
                </div>
                """))
                # Try to delete docs manually as fallback
                doc_keys = list(client.scan_iter("doc:*"))
                if doc_keys:
                    for key in doc_keys:
                        client.delete(key)
                    display(HTML(f"""
                    <div style='background: #dcfce7; border-left: 4px solid #10b981; 
                                padding: 15px; border-radius: 5px; margin: 10px 0;'>
                        <div style='color: #10b981; font-weight: bold;'>
                            ✅ Deleted {len(doc_keys)} document keys manually
                        </div>
                    </div>
                    """))
        
        else:
            display(HTML(f"""
            <div style='background: #fee2e2; border-left: 4px solid #ef4444; 
                        padding: 15px; border-radius: 5px; margin: 10px 0;'>
                <div style='color: #ef4444; font-weight: bold;'>
                    ❌ Invalid cleanup type: {cleanup_type}
                </div>
                <div style='color: #64748b; margin-top: 10px;'>
                    Valid options: 'all', 'index', 'docs', 'index_and_docs'
                </div>
            </div>
            """))
        
        # Show updated status
        display(HTML("""
        <div style='background: #f8fafc; padding: 15px; border-radius: 10px; margin: 20px 0;'>
            <h4 style='color: #475569; margin-top: 0;'>📊 Updated Database Status:</h4>
        </div>
        """))
        check_redis_health(client)
        
    except Exception as e:
        display(HTML(f"""
        <div style='background: #fee2e2; border-left: 4px solid #ef4444; 
                    padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <div style='color: #ef4444; font-weight: bold;'>
                ❌ Error during cleanup: {e}
            </div>
        </div>
        """))


# Display instructions
display(HTML("""
<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            padding: 20px; border-radius: 10px; margin: 20px 0;'>
    <h3 style='color: white; margin-top: 0;'>🔧 Health Check & Cleanup Functions Loaded</h3>
    <div style='background: rgba(255,255,255,0.1); padding: 15px; border-radius: 5px; 
                color: white;'>
        <p style='margin: 5px 0;'>✅ Functions are now available. Run one of these commands:</p>
    </div>
    <div style='background: rgba(255,255,255,0.1); padding: 15px; border-radius: 5px; 
                color: white; font-family: monospace; margin-top: 10px;'>
        <div style='margin: 10px 0;'><strong style='color: #fbbf24;'>check_redis_health(redis_client)</strong> 
            <span style='opacity: 0.8;'># Check database status</span></div>
    </div>
    <div style='background: rgba(255,255,255,0.2); padding: 10px; border-radius: 5px; 
                margin-top: 15px; color: white;'>
        <strong>💡 Cleanup Options:</strong>
    </div>
    <div style='background: rgba(255,255,255,0.1); padding: 15px; border-radius: 5px; 
                color: white; font-family: monospace; margin-top: 5px;'>
        <div style='margin: 5px 0;'><strong>cleanup_redis(redis_client, 'index_and_docs')</strong> 
            <span style='opacity: 0.8;'># ⭐ Clean slate</span></div>
        <div style='margin: 5px 0;'><strong>cleanup_redis(redis_client, 'index')</strong> 
            <span style='opacity: 0.8;'># Delete index only</span></div>
        <div style='margin: 5px 0;'><strong>cleanup_redis(redis_client, 'docs')</strong> 
            <span style='opacity: 0.8;'># Delete docs only</span></div>
        <div style='margin: 5px 0;'><strong>cleanup_redis(redis_client, 'all')</strong> 
            <span style='opacity: 0.8;'># ⚠️ Nuclear option</span></div>
    </div>
</div>
"""))

print("✅ Health check and cleanup functions ready!")

✅ Health check and cleanup functions ready!


## Create Search Index

Define the schema and create a RediSearch index for vector similarity search using COSINE distance on 1536-dimensional embeddings.

In [6]:
# Import RediSearch classes for index creation
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.field import TextField, VectorField

# Step 1: Define constants for our index
print("📋 Setting up index parameters...")

VECTOR_DIM = len(article_df['title_vector'][0])  # length of the vectors (1536)
VECTOR_NUMBER = len(article_df)                   # initial number of vectors (2500)
INDEX_NAME = "embeddings-index"                   # name of the search index
PREFIX = "doc"                                    # prefix for the document keys
DISTANCE_METRIC = "COSINE"                        # distance metric for the vectors

print(f"  Vector dimensions: {VECTOR_DIM}")
print(f"  Number of vectors: {VECTOR_NUMBER}")
print(f"  Index name: {INDEX_NAME}")
print(f"  Distance metric: {DISTANCE_METRIC}")

# Step 2: Define the schema (what fields exist and how to index them)
print("\n🏗️  Defining index schema...")

# Text fields (can be searched with keywords)
title = TextField(name="title")
url = TextField(name="url")
text = TextField(name="text")

# Vector fields (can be searched by similarity)
title_embedding = VectorField("title_vector",
    "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC,
        "INITIAL_CAP": VECTOR_NUMBER,
    }
)
text_embedding = VectorField("content_vector",
    "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC,
        "INITIAL_CAP": VECTOR_NUMBER,
    }
)

fields = [title, url, text, title_embedding, text_embedding]
print(f"  Defined {len(fields)} fields (3 text + 2 vector)")

# Step 3: Create the index (or check if it already exists)
print("\n🔨 Creating search index...")
try:
    redis_client.ft(INDEX_NAME).info()
    print(f"⚠️  Index '{INDEX_NAME}' already exists - skipping creation")
    print("   💡 Run cleanup_redis(redis_client, 'index_and_docs') to start fresh")
except:
    # Create RediSearch Index
    redis_client.ft(INDEX_NAME).create_index(
        fields = fields,
        definition = IndexDefinition(prefix=[PREFIX], index_type=IndexType.HASH)
    )
    print(f"✅ Index '{INDEX_NAME}' created successfully!")

📋 Setting up index parameters...
  Vector dimensions: 1536
  Number of vectors: 100
  Index name: embeddings-index
  Distance metric: COSINE

🏗️  Defining index schema...
  Defined 5 fields (3 text + 2 vector)

🔨 Creating search index...
⚠️  Index 'embeddings-index' already exists - skipping creation
   💡 Run cleanup_redis(redis_client, 'index_and_docs') to start fresh


## Load Documents into Index

Convert vectors to binary format and load all 100 documents into Redis. Takes ~5 seconds.

<details>
<summary>💡 <b>What's happening here?</b> (click to expand)</summary>

<br>

**What we're doing:**
1. Loop through all 100 Wikipedia articles
2. Convert vectors from Python lists → binary bytes (Redis storage format)
3. Store each article as a HASH in Redis with key `doc:1`, `doc:2`, etc.

**Why convert to bytes?**
- Redis stores binary data more efficiently than text
- A list of 1536 floats as text "[0.1, 0.2, ...]" takes more space than binary bytes

**HASH data type** = Like a Python dictionary in Redis:
```
doc:1 → {title: "April", text: "April is...", title_vector: <binary>, ...}
```

</details>

In [7]:
# Define function to load documents into Redis
def index_documents(client: redis.Redis, prefix: str, documents: pd.DataFrame):
    """
    Load a pandas DataFrame of documents into Redis.
    
    Args:
        client: Redis client connection
        prefix: Prefix for document keys (e.g., 'doc' creates 'doc:1', 'doc:2', ...)
        documents: DataFrame with columns including vectors
    """
    records = documents.to_dict("records")
    total = len(records)
    
    print(f"📝 Loading {total} documents into Redis...")
    
    for i, doc in enumerate(records):
        # Create unique key for this document
        key = f"{prefix}:{str(doc['id'])}"

        # Convert vectors from Python lists to binary format
        # Redis stores bytes more efficiently than text representations
        title_embedding = np.array(doc["title_vector"], dtype=np.float32).tobytes()
        content_embedding = np.array(doc["content_vector"], dtype=np.float32).tobytes()

        # Replace list of floats with byte vectors
        doc["title_vector"] = title_embedding
        doc["content_vector"] = content_embedding

        # Store document as a HASH in Redis
        client.hset(key, mapping = doc)
        
        # Progress indicator every 500 documents
        if (i + 1) % 500 == 0:
            print(f"  ✓ Loaded {i + 1}/{total} documents...")
    
    print(f"✅ All {total} documents loaded successfully!")

# Execute the loading
index_documents(redis_client, PREFIX, article_df)

# Verify the load
info = redis_client.info()
total_keys = info.get('db0', {}).get('keys', 0)
print(f"\n📊 Final status: {total_keys} total keys in Redis")
print(f"   Index name: {INDEX_NAME}")

📝 Loading 100 documents into Redis...


✅ All 100 documents loaded successfully!

📊 Final status: 100 total keys in Redis
   Index name: embeddings-index


## Semantic Search

Search by meaning, not just keywords. Query text is converted to a vector and matched against stored documents using K-Nearest Neighbors (KNN).

In [8]:
def search_redis(
    redis_client: redis.Redis,
    user_query: str,
    index_name: str = "embeddings-index",
    vector_field: str = "title_vector",
    return_fields: list = ["title", "url", "text", "vector_score"],
    hybrid_fields = "*",
    k: int = 20,
) -> List[dict]:

    # Creates embedding vector from user query
    embedded_query = openai.Embedding.create(input=user_query,
                                            model=EMBEDDING_MODEL,
                                            )["data"][0]['embedding']

    # Prepare the Query
    base_query = f'{hybrid_fields}=>[KNN {k} @{vector_field} $vector AS vector_score]'
    query = (
        Query(base_query)
         .return_fields(*return_fields)
         .sort_by("vector_score")
         .paging(0, k)
         .dialect(2)
    )
    params_dict = {"vector": np.array(embedded_query).astype(dtype=np.float32).tobytes()}

    # perform vector search
    results = redis_client.ft(index_name).search(query, params_dict)
    for i, article in enumerate(results.docs):
        score = 1 - float(article.vector_score)
        print(f"{i}. {article.title} (Score: {round(score ,3) })")
    return results.docs

In [9]:
# OpenAI API key is already set from config file in the setup cell above
# If you need to override it, you can uncomment the line below:
# openai.api_key = os.getenv("OPENAI_API_KEY", "your-key-here")

results = search_redis(redis_client, 'modern art in Europe', k=10)

0. Angel (Score: 0.007)
1. Argentina (Score: 0.006)
2. A (Score: 0.003)
3. Christian (Score: 0.003)
4. Animal (Score: -0.002)
5. Adobe Illustrator (Score: -0.002)
6. Abrahamic religion (Score: -0.003)
7. Architecture (Score: -0.005)
8. August (Score: -0.005)
9. Native American (Score: -0.005)


In [10]:
results = search_redis(redis_client, 'Famous battles in Scottish history', vector_field='content_vector', k=10)

0. Beijing (Score: 0.012)
1. Civics (Score: 0.011)
2. Computer science (Score: 0.008)
3. Alan Turing (Score: 0.007)
4. August (Score: 0.005)
5. April (Score: 0.004)
6. Census of Marine Life (Score: 0.002)
7. China (Score: 0.002)
8. Beekeeping (Score: 0.0)
9. Country (Score: 0.0)


## Hybrid Queries with Redis

The previous examples showed how run vector search queries with RediSearch. In this section, we will show how to combine vector search with other RediSearch fields for hybrid search. In the below example, we will combine vector search with full text search.

In [11]:
def create_hybrid_field(field_name: str, value: str) -> str:
    return f'@{field_name}:"{value}"'

In [12]:
# search the content vector for articles about famous battles in Scottish history and only include results with Scottish in the title
results = search_redis(redis_client,
                       "Famous battles in Scottish history",
                       vector_field="title_vector",
                       k=5,
                       hybrid_fields=create_hybrid_field("title", "Scottish")
                       )

In [13]:
# run a hybrid query for articles about Art in the title vector and only include results with the phrase "Leonardo da Vinci" in the text
results = search_redis(redis_client,
                       "Art",
                       vector_field="title_vector",
                       k=5,
                       hybrid_fields=create_hybrid_field("text", "Leonardo da Vinci")
                       )

# find specific mention of Leonardo da Vinci in the text that our full-text-search query returned
mention = [sentence for sentence in results[0].text.split("\n") if "Leonardo da Vinci" in sentence][0]
mention

0. Angel (Score: 0.007)
1. Art (Score: -0.016)


'The same cherubim creatures were said to be cast in gold on top of the Ark of the Covenant. Casting metal is one of the oldest forms of artwork, and was attempted by Leonardo da Vinci.'

For more example with Redis as a vector database, see the README and examples within the ``vector_databases/redis`` directory of this repository